In [ ]:
import numpy as np
import pandas as pd
import gzip
import json

from pprint import pprint
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train_classification_df = pd.read_csv("drive/MyDrive/CS412_project/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]

In [ ]:
# stats about the labels
train_classification_df.groupby("category").count()

,user_id
category,
art,191
entertainment,323
fashion,299
food,511
gaming,13
health and lifestyle,503
mom and children,149
sports,113
tech,346


In [ ]:
username2_category["kod8net"]

'tech'

In [ ]:
train_data_path = "drive/MyDrive/CS412_project/training-dataset.jsonl"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()


with open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"]
    username = profile["username"]
    if username in username2_category:
      # train data info
      username2posts_train[username] = sample["posts"]
      username2profile_train[username] = profile


    else:
      # it is test data info
      username2posts_test[username] = sample["posts"]
      username2profile_test[username] = profile


In [ ]:
# Profile Dataframe
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)

pd.set_option('display.max_columns', None)
train_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,is_verified,highlight_reel_count,bio_links,entities,ai_agent_type,fb_profile_biolink,restricted_by_viewer,country_block,eimu_id,external_url,fbid,has_clips,hide_like_and_view_counts,is_professional_account,is_supervision_enabled,is_guardian_of_viewer,is_supervised_by_viewer,is_supervised_user,is_embeds_disabled,is_joined_recently,business_address_json,business_contact_method,business_email,business_phone_number,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,deparmedya,3170700063,Depar Medya,#mediaplanning #mediabuying #sosyalmedya,Local business,None,1167,192,True,False,False,6,"""[{'title': '', 'lynx_url': 'https://l.instagr...",#mediaplanning #mediabuying #sosyalmedya,None,None,None,False,110133017051179,http://www.deparmedya.com/,17841403241896337,True,False,True,False,False,False,False,False,False,"""{\""city_name\"": null, \""city_id\"": null, \""la...",CALL,None,None,None,None,LOCAL,False,False,https://instagram.fsaw2-3.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,kafesfirin,266439571,KAFES FIRIN,📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...,Brand,None,11997,17,True,False,False,15,"""[{'title': '', 'lynx_url': 'https://l.instagr...",📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...,None,None,None,False,111783696881731,http://www.kafesfirin.com/,17841400227134163,True,False,True,False,False,False,False,False,False,"""{\""city_name\"": null, \""city_id\"": null, \""la...",CALL,None,None,None,None,BRAND,False,False,https://instagram.fada1-13.fna.fbcdn.net/v/t51...,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [ ]:
pd.set_option('display.max_columns', 10)
test_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,beyazyakaliyiz,8634457436,Selam Beyaz Yakalı,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,Personal blog,None,1265,665,True,False,...,None,None,PERSONAL_BLOG,False,False,https://instagram.fist6-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,totalenergies_istasyonlari,7066643793,TotalEnergies İstasyonları,TotalEnergies İstasyonları resmi Instagram hes...,Energy Company,None,28025,4,True,False,...,None,None,ENERGY_COMPANY,False,False,https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 8.4 MB/s eta 0:00:00


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import emoji

def preprocess_text(text: str):
    # lower casing Turkish Text, Don't use str.lower :)
    text = text.casefold()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Remove special characters and punctuation
    text = ''.join(char for char in text if char.isalnum() or char.isspace() or char in emoji.EMOJI_DATA)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


corpus = []

# to keep the label order
train_usernames = []

train_business_categories = []
train_category_enums = []

for username, posts in username2posts_train.items():
  train_usernames.append(username)

  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)

  profile = username2profile_train[username]

  bio = profile.get("biography", "")
  if (bio is not None) and (bio != ""):
    cleaned_captions.append(preprocess_text(bio))

  business_category = profile.get("business_category_name", "None")
  if business_category is None:
    business_category = "None"
  train_business_categories.append(business_category)

  category_enum = profile.get("category_enum", "NONE")
  if category_enum is None:
    category_enum = "NONE"
  train_category_enums.append(category_enum)

  # joining the posts of each user with a \n
  user_post_captions = "\n".join(cleaned_captions)
  corpus.append(user_post_captions)



In [ ]:
vectorizer = TfidfVectorizer(
    stop_words=turkish_stopwords,
    max_features=5000,
    ngram_range=(1, 3),
    tokenizer=lambda text: re.findall(r'\w+|[^\s\w]', text)
    )

# fit the vectorizer
vectorizer.fit(corpus)


# transform the data into vectors
x_post_train = vectorizer.transform(corpus)
y_train = [username2_category.get(uname, "NA") for uname in train_usernames]


test_usernames = []
test_corpus = []
test_business_categories = []
test_category_enums = []

for username, posts in username2posts_test.items():
  test_usernames.append(username)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)

    profile = username2profile_train.get(username, dict())

    bio = profile.get("biography", "")
    if (bio is not None) and (bio != ""):
      cleaned_captions.append(preprocess_text(bio))

    business_category = profile.get("business_category_name", "None")
    if business_category is None:
      business_category = "None"
    test_business_categories.append(business_category)

    category_enum = profile.get("category_enum", "NONE")
    if category_enum is None:
      category_enum = "NONE"
    test_category_enums.append(category_enum)

  user_post_captions = "\n".join(cleaned_captions)
  test_corpus.append(user_post_captions)


# Just transforming! No Fitting!!!!!
x_post_test = vectorizer.transform(test_corpus)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# Making sure everything is fine
assert y_train.count("NA") == 0

In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['a', 'a great', 'a new', ..., '\U0001faf6', '\U0001faf6 🏻',
       '\U0001faf6 🏼'], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder

df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)

# business_category_label_encoder = LabelEncoder()
# category_enum_label_encoder = LabelEncoder()

# business_category_label_encoder.fit(train_business_categories)
# category_enum_label_encoder.fit(train_category_enums)

# df_tfidf["encoded_business_category"] = business_category_label_encoder.transform(train_business_categories)
# df_tfidf["encoded_category_enum"] = category_enum_label_encoder.transform(train_category_enums)

pd.set_option('display.max_columns', None)
df_tfidf.head(2)

a  a great  a new  a unique  abdullah  abone  about  acele  acil  \
0  0.0      0.0    0.0       0.0       0.0    0.0    0.0    0.0   0.0   
1  0.0      0.0    0.0       0.0       0.0    0.0    0.0    0.0   0.0   

   acil şifalar  acı  ada  adam  adana  aday  add  adet  adet yumurta  adeta  \
0           0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0           0.0    0.0   
1           0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0           0.0    0.0   

   adidas     adlı  adres  adresi  adresinden  adresini  adresini ziyaret  \
0     0.0  0.05505    0.0     0.0         0.0       0.0               0.0   
1     0.0  0.00000    0.0     0.0         0.0       0.0               0.0   

   adventure  adı  adım  adım adım  adımlar     adına  adını  aegean  \
0        0.0  0.0   0.0        0.0      0.0  0.044351    0.0     0.0   
1        0.0  0.0   0.0        0.0      0.0  0.000000    0.0     0.0   

   aesthetic  afad  afet  afiyet  afiyet olsun  afiyetle  after  afyon  again  \
0        0.0   0.0   0.0     0.0           0.0       0.0    0.0    0.0    0.0   
1        0.0   0.0   0.0     0.0           0.0       0.0    0.0    0.0    0.0   

   agency  ahmet  ahşap  aile  ailesi  ailesi olarak  ailesine  ailesinin  \
0     0.0    0.0    0.0   0.0     0.0            0.0       0.0        0.0   
1     0.0    0.0    0.0   0.0     0.0            0.0       0.0        0.0   

   air  ait  ajans   ak  ak parti  akademi  akademik  akdeniz  aksesuar  \
0  0.0  0.0    0.0  0.0       0.0      0.0       0.0      0.0       0.0   
1  0.0  0.0    0.0  0.0       0.0      0.0       0.0      0.0       0.0   

   aktif  aktivite  akü  akıl    akıllı  akşam  akşam yemeği  akşamyemeği  \
0    0.0       0.0  0.0   0.0  0.107479    0.0           0.0          0.0   
1    0.0       0.0  0.0   0.0  0.000000    0.0           0.0          0.0   

   akşamı   al  alabilirsiniz  alacak  alalım      alan  alana  alanda  \
0     0.0  0.0            0.0     0.0     0.0  0.033167    0.0     0.0   
1     0.0  0.0            0.0     0.0     0.0  0.000000    0.0     0.0   

   alanlarda  alanları  alanlarında  alanya  alanı  alanında  alanında uzman  \
0        0.0       0.0          0.0     0.0    0.0       0.0             0.0   
1        0.0       0.0          0.0     0.0    0.0       0.0             0.0   

   alarak  alaçatı  aldı  aldık  aldım  aldığı  aldığımız  ali  alkol  all  \
0     0.0      0.0   0.0    0.0    0.0     0.0        0.0  0.0    0.0  0.0   
1     0.0      0.0   0.0    0.0    0.0     0.0        0.0  0.0    0.0  0.0   

   all our  all the  allah  allahtan  allahtan rahmet  allahın  alma  almak  \
0      0.0      0.0    0.0       0.0              0.0      0.0   0.0    0.0   
1      0.0      0.0    0.0       0.0              0.0      0.0   0.0    0.0   

   almanya  almaya  almış  alsancak  also  alt  alternatif  altı  altın  \
0      0.0     0.0    0.0       0.0   0.0  0.0         0.0   0.0    0.0   
1      0.0     0.0    0.0       0.0   0.0  0.0         0.0   0.0    0.0   

   altına  altında  always  alıcı  alın  alınan  alıp  alır  alırken  alıyor  \
0     0.0      0.0     0.0    0.0   0.0     0.0   0.0   0.0      0.0     0.0   
1     0.0      0.0     0.0    0.0   0.0     0.0   0.0   0.0      0.0     0.0   

   alıyoruz  alışveriş  amacı  amacıyla  amazing  amazingdestination  \
0       0.0        0.0    0.0  0.048344      0.0                 0.0   
1       0.0        0.0    0.0  0.000000      0.0                 0.0   

   amazingdestination familyhotel  amazingplaces  \
0                             0.0            0.0   
1                             0.0            0.0   

   amazingplaces keepcalmandtravel  \
0                              0.0   
1                              0.0   

   amazingplaces keepcalmandtravel amazingdestination  amaçlı  amaçlıdır  \
0                                                0.0      0.0        0.0   
1                                                0.0      0.0        0.0   

   ambalaj  ameliyat  amsterdam   an  a

In [ ]:
pd.set_option('display.max_columns', 10)
df_tfidf.shape

(2741, 5000)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(df_tfidf, y_train, test_size=0.2, stratify=y_train)

In [ ]:
x_train.shape

(2192, 5000)

In [ ]:
x_val.shape

(549, 5000)

In [ ]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB, GaussianNB, CategoricalNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

model = ComplementNB()
model.fit(x_train, y_train)

ComplementNB()

In [ ]:
y_train_pred = model.predict(x_train)

print("Accuracy:", accuracy_score(y_train, y_train_pred))
print("\nClassification Report:")
print(classification_report(y_train, y_train_pred, zero_division=0))

Accuracy: 0.7554744525547445

Classification Report:
                      precision    recall  f1-score   support

                 art       0.93      0.37      0.53       153
       entertainment       0.71      0.57      0.63       258
             fashion       0.72      0.90      0.80       239
                food       0.85      0.90      0.88       409
              gaming       1.00      0.10      0.18        10
health and lifestyle       0.69      0.79      0.73       402
    mom and children       0.90      0.55      0.68       119
              sports       0.95      0.69      0.80        90
                tech       0.74      0.88      0.80       277
              travel       0.68      0.76      0.71       235

            accuracy                           0.76      2192
           macro avg       0.82      0.65      0.68      2192
        weighted avg       0.77      0.76      0.75      2192



In [ ]:
y_val_pred = model.predict(x_val)

print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, zero_division=0))

Accuracy: 0.599271402550091

Classification Report:
                      precision    recall  f1-score   support

                 art       0.44      0.11      0.17        38
       entertainment       0.47      0.42      0.44        65
             fashion       0.59      0.73      0.66        60
                food       0.83      0.88      0.85       102
              gaming       0.00      0.00      0.00         3
health and lifestyle       0.57      0.65      0.61       100
    mom and children       0.62      0.27      0.37        30
              sports       0.57      0.17      0.27        23
                tech       0.57      0.71      0.63        69
              travel       0.47      0.64      0.55        59

            accuracy                           0.60       549
           macro avg       0.51      0.46      0.45       549
        weighted avg       0.59      0.60      0.57       549



In [ ]:
# let's take a look at the first 5 lines of the file
test_data_path = "drive/MyDrive/CS412_project/test-classification-round2.dat"
!head -n 5 "$test_data_path"

print("*****")

test_unames = []
with open(test_data_path, "rt") as fh:
  for line in fh:
    test_unames.append(line.strip())

print(test_unames[:5])

schirogomme
takviyegiller
egeasalmetal_
expomedeurasia
izmirhim
*****
['schirogomme', 'takviyegiller', 'egeasalmetal_', 'expomedeurasia', 'izmirhim']


In [ ]:
x_test = []

for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    x_test.append(x_post_test[index].toarray()[0])
  except Exception as e:
    try:
      index = train_usernames.index(uname)
      x_test.append(x_post_train[index].toarray()[0])
    except Exception as e:
      print(uname)

In [ ]:
df_test = pd.DataFrame(np.array(x_test), columns=feature_names)
df_test.head(2)

,a,a great,a new,a unique,abdullah,...,🩷,🫠,🫶,🫶 🏻,🫶 🏼
0,0.099264,0.0,0.0,0.0,0.0,...,0.0,0.0,0.008221,0.009984,0.0
1,0.008479,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0


In [ ]:
test_pred = model.predict(df_test)

output = dict()
for index, uname in enumerate(test_unames):
  output[uname] = test_pred[index]

In [ ]:
with open("drive/MyDrive/CS412_project/prediction-classification-round2.json", "w") as of:
  json.dump(output, of, indent=4)

In [ ]:
from datetime import datetime
def time_in_seconds(timestamp):
  dt = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
  return dt.hour * 3600 + dt.minute * 60 + dt.second

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import svm

username2regressors = dict()

def predict_like_count(username, current_post):

  global username2regressors

  if username in username2regressors:
    regressor = username2regressors[username]

    comment_count = current_post.get("comments_count", 0)

    time = time_in_seconds(current_post["timestamp"])
    day  = datetime.strptime(current_post["timestamp"], "%Y-%m-%d %H:%M:%S").weekday()

    sin_time = np.sin(2 * np.pi * time / 86400)
    cos_time = np.cos(2 * np.pi * time / 86400)
    sin_day = np.sin(2 * np.pi * day / 7)
    cos_day = np.cos(2 * np.pi * day / 7)

    result = regressor.predict([[comment_count, sin_time, cos_time, sin_day, cos_day]])[0]
    return max(result, 0)

  # Otherwise...

  posts = []
  if username in username2posts_train:
    posts = username2posts_train[username]
  elif username in username2posts_test:
    posts = username2posts_test[username]
  else:
    print(f"No data available for {username}")
    return 0

  if not posts:
    print(f"No data available for {username}")
    return 0

  comment_counts = [post.get("comments_count", 0) if post.get("comments_count", 0) is not None else 0 for post in posts]
  like_counts    = [post.get("like_count", 0) if post.get("like_count", 0) is not None else 0 for post in posts]

  times = np.array([time_in_seconds(post["timestamp"]) if post["timestamp"] is not None else 0 for post in posts])
  days  = np.array([datetime.strptime(post["timestamp"], "%Y-%m-%d %H:%M:%S").weekday() if post["timestamp"] is not None else 0 for post in posts])



  sin_times = np.sin(2 * np.pi * times / 86400)
  cos_times = np.cos(2 * np.pi * times / 86400)
  sin_days = np.sin(2 * np.pi * times / 7)
  cos_days = np.cos(2 * np.pi * times / 7)

  x_train = list(zip(comment_counts, sin_times, cos_times, sin_days, cos_days))
  y_train = like_counts

  regressor = svm.SVR()
  regressor.fit(x_train, y_train)

  username2regressors[username] = regressor

  comment_count = current_post.get("comments_count", 0)

  time = time_in_seconds(current_post["timestamp"])
  day  = datetime.strptime(current_post["timestamp"], "%Y-%m-%d %H:%M:%S").weekday()

  sin_time = np.sin(2 * np.pi * time / 86400)
  cos_time = np.cos(2 * np.pi * time / 86400)
  sin_day = np.sin(2 * np.pi * day / 7)
  cos_day = np.cos(2 * np.pi * day / 7)

  result = regressor.predict([[comment_count, sin_time, cos_time, sin_day, cos_day]])[0]
  return max(result, 0)

In [ ]:
def log_mse_like_counts(y_true, y_pred):
  """
  Calculate the Log Mean Squared Error (Log MSE) for like counts (log(like_count + 1)).

  Parameters:
  - y_true: array-like, actual like counts
  - y_pred: array-like, predicted like counts

  Returns:
  - log_mse: float, Log Mean Squared Error
  """
  # Ensure inputs are numpy arrays
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)

  # Log transformation: log(like_count + 1)
  log_y_true = np.log1p(y_true)
  log_y_pred = np.log1p(y_pred)

  # Compute squared errors
  squared_errors = (log_y_true - log_y_pred) ** 2

  # Return the mean of squared errors
  return np.mean(squared_errors)

In [ ]:
y_like_count_train_true = []
y_like_count_train_pred = []

for uname, posts in username2posts_train.items():
  for post in posts:
    pred_val = predict_like_count(uname, post)
    true_val = post.get("like_count", 0)
    if true_val is None:
      true_val = 0

    y_like_count_train_true.append(true_val)
    y_like_count_train_pred.append(pred_val)

print(f"Log MSE Train = {log_mse_like_counts(y_like_count_train_true, y_like_count_train_pred)}")

Log MSE Train = 0.7946997339108318


In [ ]:
log_mse_train = log_mse_like_counts(y_like_count_train_true, y_like_count_train_pred)

print(f"Log MSE Train = {log_mse_train}")
print(f"Log MSE Sum Train = {log_mse_train * 3000}")

Log MSE Train = 0.7946997339108318
Log MSE Sum Train = 2384.0992017324957


In [ ]:
from sklearn.metrics import mean_squared_error
mse_train = mean_squared_error(y_like_count_train_true, y_like_count_train_pred)
print(f"MSE Train = {mse_train}")

MSE Train = 770623802.9586772


In [ ]:
y_like_count_test_true = []
y_like_count_test_pred = []

for uname, posts in username2posts_test.items():
  for post in posts:
    pred_val = predict_like_count(uname, post)
    true_val = post.get("like_count", 0)
    if true_val is None:
      true_val = 0

    y_like_count_test_true.append(true_val)
    y_like_count_test_pred.append(pred_val)

print(f"Log MSE Test = {log_mse_like_counts(y_like_count_test_true, y_like_count_test_pred)}")

Log MSE Test = 0.5598439498889954


In [ ]:
log_mse_test = log_mse_like_counts(y_like_count_test_true, y_like_count_test_pred)

print(f"Log MSE Train = {log_mse_test}")
print(f"Log MSE Sum Train = {log_mse_test * 3000}")

Log MSE Train = 0.5598439498889954
Log MSE Sum Train = 1679.5318496669863


In [ ]:
path = "drive/MyDrive/CS412_project/test-regression-round2.jsonl"
output_path = "drive/MyDrive/CS412_project/prediction-regression-round2.json"

to_predict_like_counts_usernames = []
output = dict()
with open(path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    # let's predict
    pred_val = predict_like_count(sample["username"], sample)

    output[sample["id"]] = round(pred_val)

with open(output_path, "wt") as of:
  json.dump(output, of)

No data available for belediyesikose
